<a href="https://colab.research.google.com/github/LordRelentless/NGFTSimulations/blob/main/Simulation_1_1_The_Duality_of_Entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display # Import necessary modules for displaying GIF
import os # Import os module to check file

print("Running NGFT Duality of Entropy Simulation (V4)...")
print("Phase 1: Calculating physics data...")

# --- Simulation Parameters (Unchanged) ---
N_PARTICLES = 50
BOX_SIZE = 20.0
DT = 0.01
SIM_STEPS = 1500
COOLING_RATE = 0.998
INFO_COST_PER_INTERACTION = 0.2
MIN_INIT_DIST = 1.0
epsilon = 1.0
sigma = 1.0

# --- Initialization and Physics Functions (Unchanged) ---
def create_safe_initial_positions(n, box_size, min_dist):
    positions = np.zeros((n, 2))
    for i in range(n):
        attempts = 0
        while True:
            attempts += 1
            if attempts > 1000: raise RuntimeError("Could not place particles.")
            candidate_pos = np.random.rand(2) * box_size
            if i == 0: positions[i] = candidate_pos; break
            is_safe = True
            for j in range(i):
                if np.linalg.norm(candidate_pos - positions[j]) < min_dist: is_safe = False; break
            if is_safe: positions[i] = candidate_pos; break
    return positions

def calculate_forces_and_info_cost(positions):
    forces = np.zeros_like(positions)
    interaction_count = 0
    for i in range(N_PARTICLES):
        for j in range(i + 1, N_PARTICLES):
            rij = positions[i] - positions[j]
            rij -= BOX_SIZE * np.round(rij / BOX_SIZE)
            dist_sq = np.sum(rij**2)
            if dist_sq < (2.5 * sigma)**2:
                # Add a small epsilon to dist_sq to prevent division by zero
                if dist_sq < 1e-9: # Check if distance is practically zero
                    dist_sq = 1e-9 # Set a minimum distance squared to avoid division by zero
                if dist_sq < (1.5 * sigma)**2: interaction_count += 1
                inv_dist_sq = 1.0 / dist_sq
                inv_dist_6 = inv_dist_sq**3
                inv_dist_12 = inv_dist_6**2
                force_mag = 48 * epsilon * (inv_dist_12 - 0.5 * inv_dist_6) * inv_dist_sq
                # Check for NaN or Inf in force_mag and skip if found
                if np.isfinite(force_mag):
                    forces[i] += force_mag * rij
                    forces[j] -= force_mag * rij
    info_cost = interaction_count * INFO_COST_PER_INTERACTION
    return forces, info_cost

# !--- THE ONLY CHANGE IS IN THIS FUNCTION ---!
def calculate_s_mass(kinetic_energy):
    """
    Calculates Mass Entropy. Adds a small constant inside the log
    to prevent log(0) = -inf, ensuring numerical stability at low temps.
    """
    # This prevents the function from ever returning -inf.
    return N_PARTICLES * np.log((BOX_SIZE**2) * kinetic_energy + 1e-9)


# --- Phase 1: Data Calculation (Unchanged) ---
pos_history = np.zeros((SIM_STEPS, N_PARTICLES, 2))
entropy_history = {'steps': [], 's_mass': [], 's_info': [], 's_total': []}
ke_history = []
pos = create_safe_initial_positions(N_PARTICLES, BOX_SIZE, MIN_INIT_DIST)
vel = (np.random.rand(N_PARTICLES, 2) - 0.5) * 1.0
s_info_cumulative = 0.0

for step in range(SIM_STEPS):
    pos_history[step] = pos
    forces, info_cost_this_step = calculate_forces_and_info_cost(pos)
    s_info_cumulative += info_cost_this_step
    vel += 0.5 * forces * DT
    pos += vel * DT
    pos %= BOX_SIZE
    forces, _ = calculate_forces_and_info_cost(pos)
    vel += 0.5 * forces * DT
    vel *= COOLING_RATE
    kinetic_energy = 0.5 * np.sum(vel**2)
    s_m = calculate_s_mass(kinetic_energy)
    s_i = s_info_cumulative
    entropy_history['steps'].append(step)
    entropy_history['s_mass'].append(s_m)
    entropy_history['s_info'].append(s_i)
    entropy_history['s_total'].append(s_m + s_i)
    ke_history.append(kinetic_energy)

print("Phase 1 complete. All data calculated.")
print("Phase 2: Rendering animation...")


# --- Phase 2: Animation Setup and Execution (Modified for saving GIF and handling NaN/Inf) ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 9))
plt.style.use('dark_background')
ax1.set_title("Particle System (Cooling)"); ax1.set_xlim(0, BOX_SIZE); ax1.set_ylim(0, BOX_SIZE)
ax1.set_xticks([]); ax1.set_yticks([])
scatter = ax1.scatter(pos_history[0, :, 0], pos_history[0, :, 1], color='cyan')
temp_text = ax1.text(0.02, 0.95, '', transform=ax1.transAxes, fontsize=12, color='white', va='top')
ax2.set_title("NGFT Entropy Duality (Live)")
ax2.set_xlabel("Simulation Step"); ax2.set_ylabel("Entropy (Arbitrary Units)")
line_sm, = ax2.plot([], [], 'r-', label='S_Mass (Disorder)')
line_si, = ax2.plot([], [], 'c-', label='S_Info (Cumulative Cost)')
line_st, = ax2.plot([], [], 'w--', lw=2, label='S_Total (S_M + S_I)')

ax2.set_xlim(0, SIM_STEPS)

# Filter out NaN and Inf values before calculating min/max for y-axis limits
valid_s_mass = [x for x in entropy_history['s_mass'] if np.isfinite(x)]
valid_s_total = [x for x in entropy_history['s_total'] if np.isfinite(x)]

# Set y-axis limits only if there are valid data points
if valid_s_mass and valid_s_total:
    min_y = np.min(valid_s_mass)
    max_y = np.max(valid_s_total)
    ax2.set_ylim(min_y - abs(min_y)*0.1, max_y * 1.1)
else:
    # Set default limits or handle the case where all values are NaN/Inf
    ax2.set_ylim(-10, 10) # Example default limits, adjust as needed


ax2.legend(); ax2.grid(True, alpha=0.3)

def update_animation(frame):
    scatter.set_offsets(pos_history[frame])
    ke = ke_history[frame]
    temp_text.set_text(f'Step: {frame}/{SIM_STEPS}\nTemp (KE): {ke / N_PARTICLES:.2f}')
    line_sm.set_data(entropy_history['steps'][:frame+1], entropy_history['s_mass'][:frame+1])
    line_si.set_data(entropy_history['steps'][:frame+1], entropy_history['s_info'][:frame+1])
    line_st.set_data(entropy_history['steps'][:frame+1], entropy_history['s_total'][:frame+1])
    return scatter, temp_text, line_sm, line_si, line_st

# Create the animation object
ani = FuncAnimation(fig, update_animation, frames=SIM_STEPS, interval=20, blit=True, repeat=False)

# Save the animation as a GIF
gif_path = 'ngft_simulation_animation.gif'
print(f"Attempting to save animation to {gif_path}...")
ani.save(gif_path, writer='pillow', fps=50) # Increased fps for smoother animation
print(f"Animation saving process finished.")

# Close the plot to prevent it from displaying twice
plt.close(fig)

print(f"Phase 2 complete. Animation saved as {gif_path}.")
print("Phase 3: Displaying animation...")

# Check if the file exists and is not empty before displaying
if os.path.exists(gif_path) and os.path.getsize(gif_path) > 0:
    print(f"Successfully saved and found non-empty GIF file: {gif_path}")
    # Display the saved GIF
    display(HTML(f'<img src="{gif_path}">'))
else:
    print(f"Error: GIF file not found or is empty at {gif_path}.")
    print("The animation may not have saved correctly.")


# --- Final Analysis (Unchanged) ---
print("\n--- Simulation Complete ---")
# ... (rest of the analysis code) ...

Running NGFT Duality of Entropy Simulation (V4)...
Phase 1: Calculating physics data...
Phase 1 complete. All data calculated.
Phase 2: Rendering animation...
Attempting to save animation to ngft_simulation_animation.gif...
Animation saving process finished.
Phase 2 complete. Animation saved as ngft_simulation_animation.gif.
Phase 3: Displaying animation...
Successfully saved and found non-empty GIF file: ngft_simulation_animation.gif



--- Simulation Complete ---
